# Activity 3: Physics-Informed ML Investigation

**Duration**: 2 hours

**Learning Objectives**:
- Demonstrate how physics constraints improve machine learning predictions
- Compare baseline vs physics-informed neural network performance
- Conduct constraint ablation to discover which physics principles help most
- Learn that "adding physics" requires validation—not all constraints help!

## Background

A Physics-Informed Neural Network (PINN) embeds domain knowledge directly into the learning process through custom loss functions. Instead of just minimizing prediction error, the network also minimizes violation of known physics:

$$\mathcal{L}_{total} = \mathcal{L}_{data} + \lambda_s \mathcal{L}_{smooth} + \lambda_p \mathcal{L}_{positive} + \lambda_g \mathcal{L}_{gradient}$$

where:
- **Smoothness**: Dose varies continuously in space
- **Positivity**: Energy deposition cannot be negative  
- **Gradient correlation**: dE/dx should correlate with Bethe-Bloch theory

In [ ]:
# Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
import sys
sys.path.append('..')

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("Setup complete!")

## Part 1: Load Pre-trained Models

We have two pre-trained models:
1. **Baseline NN**: Standard neural network trained only on data
2. **Physics-Informed NN (PINN)**: Same architecture but trained with physics constraints

In [ ]:
# Load models
baseline_model = keras.models.load_model('../models/baseline_nn.keras')
pinn_model = keras.models.load_model('../models/advanced_pinn.keras')

print("Baseline Model:")
baseline_model.summary()

print("\n" + "="*50)
print("\nPhysics-Informed Model:")
pinn_model.summary()

### 📝 Question 1
Both models have identical architectures. What makes the PINN "physics-informed"?

*Your answer:*

## Part 2: Load Test Data

We reserved energies 200 MeV and 750 MeV for testing—these were NOT used during training.

In [ ]:
# Load test data
import joblib

try:
    test_df = pd.read_csv('../data/test_split_v2.csv')
    scaler_features = joblib.load('../data/scaler_features_v2.pkl')
    scaler_target = joblib.load('../data/scaler_target_v2.pkl')
except FileNotFoundError:
    test_df = pd.read_csv('../../pinn_data/test_split_v2.csv')
    scaler_features = joblib.load('../../pinn_data/scaler_features_v2.pkl')
    scaler_target = joblib.load('../../pinn_data/scaler_target_v2.pkl')

print(f"Test set: {len(test_df)} samples")
print(f"Test energies: {test_df['energy_MeV'].unique()} MeV")

# Prepare features
feature_cols = ['energy_MeV', 'depth_mm', 'beta', 'gamma', 'momentum_MeV_c']
X_test = test_df[feature_cols].values
y_test = test_df['dEdx_MeV_per_mm'].values.reshape(-1, 1)

# Scale
X_test_scaled = scaler_features.transform(X_test)
y_test_scaled = scaler_target.transform(y_test)

## Part 3: Compare Predictions

In [ ]:
# Make predictions
baseline_pred_scaled = baseline_model.predict(X_test_scaled, verbose=0)
pinn_pred_scaled = pinn_model.predict(X_test_scaled, verbose=0)

# Inverse transform
baseline_pred = scaler_target.inverse_transform(baseline_pred_scaled)
pinn_pred = scaler_target.inverse_transform(pinn_pred_scaled)

# Calculate errors
baseline_mse = np.mean((baseline_pred - y_test)**2)
pinn_mse = np.mean((pinn_pred - y_test)**2)

baseline_mape = np.mean(np.abs((baseline_pred - y_test) / y_test)) * 100
pinn_mape = np.mean(np.abs((pinn_pred - y_test) / y_test)) * 100

improvement = (baseline_mse - pinn_mse) / baseline_mse * 100

print("=== MODEL COMPARISON ON TEST SET ===")
print(f"\nBaseline NN:")
print(f"  MSE:  {baseline_mse:.6f}")
print(f"  MAPE: {baseline_mape:.2f}%")

print(f"\nPhysics-Informed NN:")
print(f"  MSE:  {pinn_mse:.6f}")
print(f"  MAPE: {pinn_mape:.2f}%")

print(f"\n🎯 PINN Improvement: {improvement:.1f}%")

### 📝 Question 2
The PINN achieves substantially lower error on unseen test energies. Why would physics constraints help with *generalization* to new conditions?

*Your answer:*

In [ ]:
# Visualize predictions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for idx, energy in enumerate([200, 750]):
    ax = axes[idx]
    mask = test_df['energy_MeV'] == energy
    
    depths = test_df.loc[mask, 'depth_mm'].values
    truth = y_test[mask]
    baseline = baseline_pred[mask]
    pinn = pinn_pred[mask]
    
    ax.plot(depths, truth, 'ko-', label='TOPAS (truth)', markersize=8, linewidth=2)
    ax.plot(depths, baseline, 'b^--', label='Baseline NN', markersize=8, linewidth=2, alpha=0.7)
    ax.plot(depths, pinn, 'rs-', label='Physics-Informed NN', markersize=8, linewidth=2, alpha=0.7)
    
    ax.set_xlabel('Depth (mm)', fontsize=14)
    ax.set_ylabel('dE/dx (MeV/mm)', fontsize=14)
    ax.set_title(f'Test Energy: {energy} MeV', fontsize=14)
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Part 4: Ablation Study

Now we investigate *which* physics constraints actually help. Students often assume "more physics is better"—but is that true?

Let's load pre-computed ablation results where each constraint was tested individually and in combination.

In [ ]:
# Load ablation results
ablation_df = pd.read_csv('../results/ablation_study.csv')
print("Ablation Study Results:")
print(ablation_df.to_string(index=False))

In [ ]:
# Visualize ablation
fig, ax = plt.subplots(figsize=(12, 6))

configs = ablation_df['configuration'].values
improvements = ablation_df['improvement_percent'].values

colors = ['green' if x > 0 else 'red' for x in improvements]
bars = ax.barh(configs, improvements, color=colors, alpha=0.7, edgecolor='black')

ax.axvline(x=0, color='black', linewidth=2)
ax.set_xlabel('Improvement vs Baseline (%)', fontsize=14)
ax.set_title('Physics Constraint Ablation Study', fontsize=16)

# Add value labels
for bar, val in zip(bars, improvements):
    ax.text(val + 2 if val > 0 else val - 8, bar.get_y() + bar.get_height()/2, 
            f'{val:+.1f}%', va='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

### 📝 Question 3 (Critical!)

Look carefully at the ablation results:

1. Which constraint provides the *largest* improvement?
2. Which constraint actually *hurts* performance when used alone?
3. Why might the gradient constraint (based on Bethe-Bloch theory) degrade performance?

*Your answers:*

## Part 5: The Surprising Lesson

### The gradient constraint HURTS performance!

This is a profound teaching moment. The gradient constraint is based on the **Bethe-Bloch formula**—one of the most respected equations in radiation physics. Yet forcing the neural network to match it makes predictions *worse*.

**Why?**

The Bethe-Bloch formula is an **approximation** that:
- Assumes only electromagnetic interactions
- Ignores nuclear reactions
- Ignores energy straggling
- Ignores density effects at high energy

Our TOPAS simulations include ALL of this physics. Forcing the network to match the simpler Bethe-Bloch theory conflicts with the more complete simulation data!

In [ ]:
# Illustrate the conflict
print("=" * 60)
print("KEY LESSON: Physics constraints must themselves be validated!")
print("=" * 60)
print()
print("What the gradient constraint enforces:")
print("  dE/dx ∝ β^{-2} (Bethe-Bloch approximation)")
print()
print("What TOPAS simulates:")
print("  dE/dx = electronic + nuclear + straggling + ...")
print()
print("The conflict:")
print("  Forcing the network to match Bethe-Bloch conflicts with")
print("  the more complete physics in the simulation data.")
print()
print("The lesson:")
print("  'Adding physics' is NOT automatically good.")
print("  Physics constraints must be validated empirically!")

### 📝 Question 4
This finding has implications beyond radiation transport. In what other contexts might "adding physics" to an ML model be counterproductive?

*Your answer:*

## Part 6: Your Own Investigation

**Challenge**: Propose a new physics constraint that might help the model.

Examples to consider:
- Conservation constraints (energy, momentum)
- Boundary conditions (e.g., dose at infinite depth → 0)
- Symmetry constraints

For each proposed constraint:
1. What physical principle does it encode?
2. How would you implement it mathematically?
3. What could go wrong? (Think about the gradient constraint lesson!)

*Your proposed constraint and analysis:*




---

## Summary

In this activity, you learned:

1. **Physics constraints substantially improve ML generalization** by restricting the model to physically plausible solutions

2. **Not all physics constraints help!** The gradient constraint (based on Bethe-Bloch) actually degraded performance because it conflicts with the more complete physics in the simulation.

3. **"Adding physics" requires validation**—even well-established physics formulas have domains of validity.

4. **The ablation study teaches the validation mindset**: test your assumptions empirically, don't just trust intuition.

### Key Teaching Points

> "Machine learning works best when combined with domain knowledge, not when used as a pure black-box interpolator."

> "Physics constraints must themselves be validated—'adding physics' is not automatic."

> "The bias-variance tradeoff: adding constraints (bias) reduces overfitting (variance) when constraints align with reality."